In [2]:
from google.cloud import bigquery
import pandas as pd

In [3]:
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: oceanic-variety-386313


In [4]:
import os
os.listdir('./data')

['alcoholic_beverages',
 '.ipynb_checkpoints',
 'logo_train_data',
 'global_warming']

## Create a new dataset

A dataset is contained within a specific [project](https://cloud.google.com/bigquery/docs/projects). Datasets are top-level containers that are used to organize and control access to your [tables](https://cloud.google.com/bigquery/docs/tables) and [views](https://cloud.google.com/bigquery/docs/views). A table or view must belong to a dataset. You need to create at least one dataset before [loading data into BigQuery](https://cloud.google.com/bigquery/loading-data-into-bigquery).

In [11]:
dir(client)

['SCOPE',
 '_SET_PROJECT',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_api',
 '_client_cert_source',
 '_connection',
 '_credentials',
 '_dataset_from_arg',
 '_default_load_job_config',
 '_default_query_job_config',
 '_determine_default',
 '_do_multipart_upload',
 '_do_resumable_upload',
 '_ensure_bqstorage_client',
 '_get_query_results',
 '_http',
 '_http_internal',
 '_initiate_resumable_upload',
 '_list_rows_from_query_results',
 '_location',
 '_schema_from_json_file_object',
 '_schema_to_json_file_object',
 'cancel_job',
 'close',
 'copy_table',
 'create_dataset',
 'create_job',
 'create_routi

In [12]:
dataset = client.dataset('reddit_db')

In [13]:
dataset

DatasetReference('oceanic-variety-386313', 'reddit_db')

In [6]:
# Define a name for the new dataset.
dataset_id = 'reddit_db'

# The project defaults to the Client's project if not specified.
dataset = client.create_dataset(dataset_id)  # API request

## raw_reddit_pull

In [6]:
df = pd.read_csv('./data/alcoholic_beverages/raw_reddit_posts.csv', index_col = 0)
df.head()

,created,created_utc,subreddit,subreddit_id,selftext,author_fullname,title,name,id,author,num_comments,url,upvote_ratio,ups,search_term
0,2023-04-26 08:37:09-05:00,1.682516e+09,therewasanattempt,t5_39ne7,NaN,t2_hjvdrg5v,to underestimate alcohol,t3_12zhtwt,12zhtwt,HornyDiggler,3471,https://v.redd.it/43hzn0d1f8wa1,0.81,56008,alcohol
1,2023-03-01 16:42:49-06:00,1.677711e+09,unpopularopinion,t5_2tk0s,"I was never a problematic drinker, but did alw...",t2_a3ih64q1,Life is better in every way when you ditch alc...,t3_11fknkh,11fknkh,Pure_Club_8997,544,https://www.reddit.com/r/unpopularopinion/comm...,0.89,2201,alcohol
2,2023-01-31 17:27:18-06:00,1.675208e+09,AskReddit,t5_2qh1i,NaN,t2_2wguso99,"Redditors, why don't you drink alcohol or why ...",t3_10qdyyd,10qdyyd,LuGGooo,2524,https://www.reddit.com/r/AskReddit/comments/10...,0.84,797,alcohol
3,2023-03-02 15:30:39-06:00,1.677793e+09,loseit,t5_2rz8w,I'm not saying you can't lose weight while dri...,t2_1uc2ctml,"If you're serious about this, take a break fro...",t3_11geo7u,11geo7u,noodlesworldwide,284,https://www.reddit.com/r/loseit/comments/11geo...,0.96,2512,alcohol
4,2023-04-13 02:12:06-05:00,1.681370e+09,kratom,t5_2qx0h,Anyone else like this combo? Kratom alone is p...,t2_9rxdm8ll,Alcohol and kratom,t3_12kfpoq,12kfpoq,slideguitarking,103,https://www.reddit.com/r/kratom/comments/12kfp...,0.78,20,alcohol


In [46]:
raw_reddit_pull_cols = ['id','created','subreddit','subreddit_id','selftext','author_fullname','title','name','url']
raw_reddit_pull = df[raw_reddit_pull_cols]
raw_reddit_pull = raw_reddit_pull[~raw_reddit_pull.duplicated()]

len(raw_reddit_pull)

4056

In [47]:
# create table
# table_ref = dataset.table("raw_reddit_pulls")
# job = client.load_table_from_dataframe(raw_reddit_pull, table_ref, location="US")

# job.result()  # Waits for table load to complete.
# print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/oceanic-variety-386313/datasets/reddit_db/tables/raw_reddit_pulls


In [5]:
query = """
    SELECT *
    FROM reddit_db.raw_reddit_pulls
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

raw_reddit_pull = query_job.to_dataframe()
raw_reddit_pull

,id,created,subreddit,subreddit_id,selftext,author_fullname,title,name,url
0,132urnj,2023-04-29 09:29:49-05:00,math,t5_2qh0n,"As my physics professor used to call them, 'co...",t2_n609s,'Cocktail Party Tricks' in Math?,t3_132urnj,https://www.reddit.com/r/math/comments/132urnj...
1,12ysjlo,2023-04-25 14:01:02-05:00,linux,t5_2qh1a,"A while ago I created a software, called [exeC...",t2_2v3gnt,Improved Wine gaming with exeCute,t3_12ysjlo,https://www.reddit.com/r/linux/comments/12ysjl...
2,133skw,2012-11-12 22:04:59-06:00,AskReddit,t5_2qh1i,I've been with this company for less than a ye...,t2_9l6f8,How to overcome shyness in work parties/happyh...,t3_133skw,https://www.reddit.com/r/AskReddit/comments/13...
3,fcalx2,2020-03-02 05:16:31-06:00,Music,t5_2qh1u,Five giants of classical music were also great...,t2_e9y211,Five classical composers that loved wine,t3_fcalx2,https://www.reddit.com/r/Music/comments/fcalx2...
4,1330lvj,2023-04-29 11:46:28-05:00,atheism,t5_2qh2p,So my gf is a Christian and she saw videos on...,t2_m8pxom06,My girlfriend broke up with me cause the holy ...,t3_1330lvj,https://www.reddit.com/r/atheism/comments/1330...
...,...,...,...,...,...,...,...,...,...
4051,1336or7,2023-04-29 16:04:43-05:00,InstaSlave,t5_8afujn,None,t2_7v840gw0,dm me if anyone want tequila premium app videos,t3_1336or7,https://www.reddit.com/gallery/1336or7
4052,13e8lce,2023-05-10 19:50:19-05:00,Ebonyadmirer,t5_8b538v,None,t2_8oqsaj7sb,That brown liquor,t3_13e8lce,https://i.redd.it/ky70i77e55za1.gif
4053,139ia96,2023-05-06 04:45:25-05:00,CockHungryGirls,t5_8cik0f,None,t2_cs13h9vs,Hot Tequila 🥵,t3_139ia96,https://redgifs.com/watch/fluffygrumpycollardl...
4054,13a97ft,2023-05-06 20:53:15-05:00,u_altarbackupnew,t5_8cv91m,None,t2_aqd2jrhhd,"HTTP.TEQUILA_ EXCLUSIVE CONTENTS, NUDE CUMSHOT...",t3_13a97ft,https://www.reddit.com/r/u_altarbackupnew/comm...


## post_exposure



In [7]:
df['measurement_date'] = 20230511
post_exposure_cols = ['id','num_comments','ups','upvote_ratio','measurement_date']

post_exposure = df[post_exposure_cols] 
post_exposure = post_exposure[~post_exposure.id.duplicated()]
post_exposure

,id,num_comments,ups,upvote_ratio,measurement_date
0,12zhtwt,3471,56008,0.81,20230511
1,11fknkh,544,2201,0.89,20230511
2,10qdyyd,2524,797,0.84,20230511
3,11geo7u,284,2512,0.96,20230511
4,12kfpoq,103,20,0.78,20230511
...,...,...,...,...,...
4201,zxlmsj,1,0,0.40,20230511
4202,meigx8,12,57,0.88,20230511
4203,n5pcpq,1,3,0.80,20230511
4204,cl90fx,0,2,1.00,20230511


In [54]:
table_ref = dataset.table("post_exposure")
job = client.load_table_from_dataframe(post_exposure, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/oceanic-variety-386313/datasets/reddit_db/tables/post_exposure


In [8]:
query = """
    SELECT *
    FROM reddit_db.post_exposure
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

post_exposure = query_job.to_dataframe()
post_exposure

,id,num_comments,ups,upvote_ratio,measurement_date
0,136jcq3,119,87,0.75,20230511
1,138ll55,152,561,0.75,20230511
2,13boorl,195,4,0.75,20230511
3,13bb0fr,51,41,0.75,20230511
4,136qu67,238,568,0.75,20230511
...,...,...,...,...,...
4051,13588ys,49,639,0.96,20230511
4052,13egpjf,49,29,0.93,20230511
4053,13a9hae,49,8,0.90,20230511
4054,1371xym,49,7,0.90,20230511


## post_search_terms

In [9]:
cols = ['id','search_term']
post_search_terms = df[cols]
post_search_terms = post_search_terms[~post_search_terms.id.duplicated()]
post_search_terms

,id,search_term
0,12zhtwt,alcohol
1,11fknkh,alcohol
2,10qdyyd,alcohol
3,11geo7u,alcohol
4,12kfpoq,alcohol
...,...,...
4201,zxlmsj,drinkresponsibly
4202,meigx8,drinkresponsibly
4203,n5pcpq,drinkresponsibly
4204,cl90fx,drinkresponsibly


In [14]:
table_ref = dataset.table("post_search_terms")
job = client.load_table_from_dataframe(post_search_terms, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/oceanic-variety-386313/datasets/reddit_db/tables/post_search_terms


# Data Enhancments

In [35]:
df_enhanced = pd.read_csv('./data/alcoholic_beverages/enhanced_reddit_posts.csv', index_col = 0)
df_enhanced.head(2)

,created,created_utc,subreddit,subreddit_id,selftext,author_fullname,title,name,id,author,...,Health and wellness tips,Advocacy and activism,Celebrity endorsements and sponsorships,Related to alcoholic beverages,Not Related to alcoholic beverages,title_vader_sentiment,selftext_vader_sentiment,overall_sentiment,alcoholic_beverages,alcoholic_beverages_score
0,2011-09-12 10:22:23-05:00,1.315841e+09,washingtondc,t5_2qi2g,NaN,t2_rcmh,"Tuesday, September 13 - Hacker News Meetup Hap...",t3_kd4v1,kd4v1,BlueOrange,...,0.030481,0.249471,0.043993,0.836798,0.163202,0.0,NaN,0.0,Hacker News Meetup Happyhour,0.001405
1,2011-12-27 07:48:34-06:00,1.324994e+09,pittsburgh,t5_2qhrn,NaN,t2_4woxy,Hey Reddit! My friend's mockumentary featuring...,t3_ns9kk,ns9kk,pghpride,...,0.010047,0.023461,0.010186,0.982127,0.017873,0.0,NaN,0.0,Pittsburgh,0.000058


## vader_sentiment

In [57]:
cols = ['id','title_vader_sentiment', 'selftext_vader_sentiment']
df_vader = df_enhanced[cols]
df_vader = df_vader[~df_vader.id.duplicated()]
df_vader

,id,title_vader_sentiment,selftext_vader_sentiment
0,kd4v1,0.0000,NaN
1,ns9kk,0.0000,NaN
2,wflb2,0.0000,NaN
3,x4kpm,0.6792,0.7835
4,133skw,-0.3182,0.4931
...,...,...,...
3764,13ewmsy,0.0000,NaN
3765,13exbks,0.0000,0.0000
3766,13exsim,-0.3400,NaN
3767,13ey3op,0.0000,NaN


In [58]:
table_ref = dataset.table('vader_sentiment')
job = client.load_table_from_dataframe(df_vader, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/oceanic-variety-386313/datasets/reddit_db/tables/vader_sentiment


## Topic Specific Tables

In [37]:
topic = 'alcoholic beverages'
topic_ = topic.lower().replace(' ','_').replace('-','_')

## topic_relavence

In [29]:
f'Related to {topic}'.lower().replace(' ','_').replace('-','_')

'related_to_alcoholic_beverages'

In [36]:
cols = [f'Related to {topic}']
new_cols = [x.lower().replace(' ','_').replace('-','_') for x in cols]
topic_relavence = df_enhanced[['id']+cols]
topic_relavence = topic_relavence.rename(columns = dict(zip(cols,new_cols)))

,id,related_to_alcoholic_beverages
0,kd4v1,0.836798
1,ns9kk,0.982127
2,wflb2,0.996331
3,x4kpm,0.946011
4,133skw,0.756050
...,...,...
3764,13ewmsy,0.996777
3765,13exbks,0.996648
3766,13exsim,0.996213
3767,13ey3op,0.996233


In [38]:
table_ref = dataset.table(f"{topic_}_relavence")
job = client.load_table_from_dataframe(topic_relavence, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/oceanic-variety-386313/datasets/reddit_db/tables/alcoholic_beverages_relavence


## topic_categories

In [46]:
categories = ['Alcohol-related news and trends', 'Wine and beer recommendations',
       'Alcohol-related memes and humor', 'Reviews of bars and restaurants',
       'Party and event photos', 'Personal stories and experiences',
       'Cocktail recipes and tutorials', 'Health and wellness tips',
       'Advocacy and activism', 'Celebrity endorsements and sponsorships']
categories_ = [x.lower().replace(' ','_').replace('-','_') for x in categories]
categories_

['alcohol_related_news_and_trends',
 'wine_and_beer_recommendations',
 'alcohol_related_memes_and_humor',
 'reviews_of_bars_and_restaurants',
 'party_and_event_photos',
 'personal_stories_and_experiences',
 'cocktail_recipes_and_tutorials',
 'health_and_wellness_tips',
 'advocacy_and_activism',
 'celebrity_endorsements_and_sponsorships']

In [45]:
cols = ['id'] + categories

topic_categories = df_enhanced[cols].rename(columns = dict(zip(categories, categories_)))
topic_categories

,id,alcohol_related_news_and_trends,wine_and_beer_recommendations,alcohol_related_memes_and_humor,reviews_of_bars_and_restaurants,party_and_event_photos,personal_stories_and_experiences,cocktail_recipes_and_tutorials,health_and_wellness_tips,advocacy_and_activism,celebrity_endorsements_and_sponsorships
0,kd4v1,0.025968,0.035920,0.058734,0.124537,0.270208,0.133934,0.026753,0.030481,0.249471,0.043993
1,ns9kk,0.145735,0.026912,0.640193,0.032375,0.061144,0.037306,0.012642,0.010047,0.023461,0.010186
2,wflb2,0.380068,0.113876,0.206374,0.053853,0.034285,0.104757,0.033896,0.025854,0.028125,0.018912
3,x4kpm,0.300236,0.100534,0.266426,0.040256,0.049769,0.063441,0.019133,0.027885,0.089966,0.042354
4,133skw,0.127635,0.049162,0.087431,0.054021,0.303125,0.110165,0.069093,0.054631,0.103943,0.040794
...,...,...,...,...,...,...,...,...,...,...,...
3764,13ewmsy,0.155797,0.007200,0.706549,0.014132,0.026556,0.032734,0.019722,0.009853,0.013373,0.014085
3765,13exbks,0.502708,0.027860,0.132798,0.036445,0.064229,0.078975,0.035869,0.030429,0.042535,0.048153
3766,13exsim,0.538508,0.016869,0.149013,0.030240,0.032626,0.090370,0.017429,0.014713,0.090757,0.019476
3767,13ey3op,0.244844,0.005795,0.612003,0.022864,0.027547,0.018057,0.034822,0.008747,0.011627,0.013695


In [47]:
table_ref = dataset.table(f"{topic_}_categories")
job = client.load_table_from_dataframe(topic_categories, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/oceanic-variety-386313/datasets/reddit_db/tables/alcoholic_beverages_categories


In [19]:
df_enhanced.columns

Index(['Unnamed: 0', 'created', 'created_utc', 'subreddit', 'subreddit_id',
       'selftext', 'author_fullname', 'title', 'name', 'id', 'author',
       'num_comments', 'url', 'upvote_ratio', 'ups', 'search_term',
       'Alcohol-related news and trends', 'Wine and beer recommendations',
       'Alcohol-related memes and humor', 'Reviews of bars and restaurants',
       'Party and event photos', 'Personal stories and experiences',
       'Cocktail recipes and tutorials', 'Health and wellness tips',
       'Advocacy and activism', 'Celebrity endorsements and sponsorships',
       'Related to alcoholic beverages', 'Not Related to alcoholic beverages',
       'title_vader_sentiment', 'selftext_vader_sentiment',
       'overall_sentiment', 'alcoholic_beverages',
       'alcoholic_beverages_score'],
      dtype='object')

In [23]:
for i, _id in df[df['id'].duplicated()].id.items():
    print(row)
    break

13bdh5n


In [31]:
# duplicates can be caused by search_terms pulling the same posts

# iterate through duplicate ids
for i, _id in df[df['id'].duplicated()].id.items():
    # make a string of the list of search terms i.e. "['drinks', 'beer', 'drinking']"
    search_terms = str(list(df[df['id'] == _id].search_term.unique()))
    # make all the posts with that id have the same search_terms
    df.loc[df[df['id'] == this_id].index,'search_term'] = search_terms



['drinks', 'beer', 'drinking']

In [35]:
df[df['id'] == this_id]['search_term'].index

Int64Index([260, 772, 4177], dtype='int64')

In [37]:
df.loc[df[df['id'] == this_id].index,'search_term'] = str(list(df[df['id'] == this_id].search_term.unique()))

In [38]:
df.loc[df[df['id'] == this_id].index,'search_term']

260     ['drinks', 'beer', 'drinking']
772     ['drinks', 'beer', 'drinking']
4177    ['drinks', 'beer', 'drinking']
Name: search_term, dtype: object

In [41]:
"['drinks', 'beer', 'drinking']".find('door')

-1